<a href="https://colab.research.google.com/github/vkt1414/Cloud-Resources-Workflows/blob/main/Notebooks/Totalsegmentator/slicerMappingsTotalSegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import json
import yaml
import os
import sys
from pathlib import Path
import time


In [3]:
curr_dir   = Path().absolute()
os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

Fri Mar 31 06:47:29 2023

Current directory :/content
Python version    : 3.9.16 (main, Dec  7 2022, 01:11:51) 


In [4]:
try:
  os.remove(f'{curr_dir}/TotalSegmentator.py')
except OSError:
  pass
!wget https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py



--2023-03-31 06:47:33--  https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63589 (62K) [text/plain]
Saving to: ‘TotalSegmentator.py’

TotalSegmentator.py 100%[===================>]  62.10K  --.-KB/s    in 0.007s  

2023-03-31 06:47:33 (8.94 MB/s) - ‘TotalSegmentator.py’ saved [63589/63589]



In [5]:
with open('TotalSegmentator.py', 'r') as f:
    content = f.read()
    tree = ast.parse(content)

class TotalSegmentatorLogicFinder(ast.NodeVisitor):
    def __init__(self):
        self.result = None

    def visit_ClassDef(self, node):
        if node.name == 'TotalSegmentatorLogic':
            for body_node in node.body:
                if isinstance(body_node, ast.FunctionDef) and body_node.name == '__init__':
                    for expr in body_node.body:
                        if isinstance(expr, ast.Assign):
                            target = expr.targets[0]
                            if isinstance(target, ast.Attribute) and target.attr == 'totalSegmentatorLabelTerminology':
                                self.result = ast.literal_eval(ast.unparse(expr.value))
        self.generic_visit(node)

finder = TotalSegmentatorLogicFinder()
finder.visit(tree)
totalSegmentatorLabelTerminology = finder.result
data = []
for key, value in totalSegmentatorLabelTerminology.items():
    fields = value.split('~')
    row = {'Structure': key}
    for i, field in enumerate(fields):
        subfields = field.split('^')
        if i == 0:
            if len(subfields) > 1:
                row['SegmentAlgorithmName'] = subfields[0]
                row['SegmentAlgorithmType'] = subfields[1]
        elif i == 1:
            if len(subfields) > 2:
                row['SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyCategoryCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyCategoryCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 2:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 3:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'] = subfields[2]
    data.append(row)

df = pd.DataFrame(data)
df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,spleen,SCT,123037004,Anatomical Structure,SCT,78961009,Spleen,,,
1,kidney_right,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,24028007,Right
2,kidney_left,SCT,123037004,Anatomical Structure,SCT,64033007,Kidney,SCT,7771000,Left
3,gallbladder,SCT,123037004,Anatomical Structure,SCT,28231008,Gallbladder,,,
4,liver,SCT,123037004,Anatomical Structure,SCT,10200004,Liver,,,
...,...,...,...,...,...,...,...,...,...,...
99,autochthon_left,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,7771000,Left
100,autochthon_right,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,24028007,Right
101,iliopsoas_left,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,7771000,Left
102,iliopsoas_right,SCT,123037004,Anatomical Structure,SCT,68455001,Iliopsoas muscle,SCT,24028007,Right
